In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [ ]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient

In [ ]:
PI_table1 = pd.DataFrame(Graph_Entropy_measure.PI_Rank, columns = ('nodes_PI','PI_i'))
#PI_table1.PI_i[PI_table.PI_i  < 0] = 0
PI_table = PI_table1.copy()
i = math.sqrt(2)
PI_table.PI_i = PI_table.PI_i.where(lambda x: x > 0, lambda x: x + i)

In [ ]:
TC_table = pd.DataFrame(Tr_centrality_measure.TC_Rank, columns = ('nodes_TC','TC_i'))
#TC_table.nodes_TC = TC_table.nodes_TC + 1
CC_table = pd.DataFrame(Constraint_Coefficient.sorted_x, columns = ('nodes_CC','CC_i'))
#CC_table.nodes_CC= CC_table.nodes_CC + 1
Result_table = pd.concat([TC_table, PI_table, CC_table], axis=1, sort=False)

First_Five= pd.DataFrame(Result_table.loc[0:2])
First_Five.index = First_Five.index + 1
First_Five

In [ ]:
#return the total power of node i.
def TPN(i):
    PI_ofi = 1/Graph_Entropy_measure.PI_of(i)
    TC_ofi = Tr_centrality_measure.TC_of(i)
    CC_ofi = Constraint_Coefficient.CC_of(i)
    #Weight to normalize the measures
    dn=PI_ofi+TC_ofi+CC_ofi
    n1=round(TC_ofi/dn, 2)
    n2=round(PI_ofi/dn, 2)
    n3=round(CC_ofi/dn, 2)
    #equation []
    TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)       
    return TPN_i

In [ ]:
#loop total power of node all i
def loop_TPN():
    i=0
    nodes = list(pd.concat([First_Five['nodes_TC'], First_Five['nodes_PI'], First_Five['nodes_CC']]).unique())
    list_TPN=[]    
    for i in nodes:          
        TPN_ = TPN(i)
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)
    return  list_TPN

In [ ]:
#Ranking the total power of node i
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1], reverse=True)

In [ ]:
#return table containing the TPN values[TPN_table and Top_Five]
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes_TPN','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:4])
Top_Five.index = Top_Five.index + 1
#TPN_table.nodes_TPN = TPN_table.nodes_TPN + 1
print(TPN_table)
#TPN_table.to_csv('result/TPN_[]_network.csv')
#Top_Five.to_csv('result/TPN_[]_network.csv')

In [ ]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Propose_Result_table.nodes_PI = Propose_Result_table.nodes_PI + 1
Propose_Result_table.nodes_TC = Propose_Result_table.nodes_TC + 1
Propose_Result_table.nodes_CC = Propose_Result_table.nodes_CC + 1
Propose_Result_table.nodes_TPN = Propose_Result_table.nodes_TPN + 1
Top_Fivep = pd.DataFrame(Propose_Result_table.loc[0:4])

#Top_Fivep.index = Top_Fivep.index + 1
#convert column nodes_TPN to int
#Top_Five.nodes_TPN= Top_Five.nodes_TPN.astype(int)
print(Top_Fivep)
#Propose_Table.to_csv('result/Propose_Table_kc_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

In [ ]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_FiveR = pd.DataFrame(Propose_Result_table.loc[0:2])
Top_FiveR.index = Top_FiveR.index + 1
#convert column nodes_TPN to int
Top_FiveR.nodes_TPN= Top_FiveR.nodes_TPN.astype(int)
ranking_table = pd.concat([Top_FiveR['nodes_TC'], Top_FiveR['nodes_PI'], Top_FiveR['nodes_CC'],Top_FiveR['nodes_TPN']], axis=1, sort=False)
print(ranking_table)
#Propose_Table.to_csv('result/Propose_Table_power_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

In [ ]:
# calculate the kendall's correlation between two variables# calcul 
from numpy.random import rand
from numpy.random import seed
from scipy.stats import kendalltau

# prepare data
data1 = Top_FiveR['TPN_i']
data2 = data1 + Top_FiveR['CC_i']+ Top_FiveR['PI_i']+ Top_FiveR['TC_i']
# calculate kendall's correlation
coef, p = kendalltau(data1, data2)
print('Kendall correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

In [ ]:
cent = pd.read_csv('result/kc/Top_Five_cent_kc_network.csv')
all_in = pd.concat([cent,Top_Fivep], axis=1, sort=False)

Top_Five_all_in = pd.DataFrame(all_in.loc[0:4])
ranking_table_all_in = pd.concat([Top_Five_all_in['nodes_tr'], Top_Five_all_in['nodes_bc'], Top_Five_all_in['nodes_cnc'],
                                  Top_Five_all_in['nodes_dc'], Top_Five_all_in['nodes_ec'],Top_Five_all_in['nodes_pr'],
                                  Top_Five_all_in['nodes_TC'], 
                                  Top_Five_all_in['nodes_PI'], Top_Five_all_in['nodes_CC'],Top_Five_all_in['nodes_TPN']], 
                                 axis=1, sort=False)

#ranking_table_all_in.to_csv('result/ranking_table_all_in_NS.csv')
ranking_table_all_in

In [ ]:
# Importing required libraries
import researchpy as rp
from scipy import stats
#The casewise method returns all information in 3 dataframes
#corr_type, corr_matrix, corr_ps=>
corr_type, corr_matrix, corr_ps = rp.corr_case(Top_Five_all_in[['TR', 'BC', 'CnC', 'DC', 'EC','pr','TC_i', 'PI_i', 'CC_i', 'TPN_i']])
#corr_type, corr_matrix, corr_ps = rp.corr_case(ranking_table_all_in[['nodes_tr', 'nodes_bc', 'nodes_cnc', 'nodes_dc', 'nodes_ec','nodes_pr','nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])
#PAIRWISEcorrelation
corr_pair = rp.corr_pair(ranking_table_all_in[['nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])

In [ ]:
corr_matrix
#corr_matrix.to_csv('result/corr_matrix_kc1.csv')

In [ ]:
#Graph_Entropy_measure.PI_of(23)